In [1]:
import wandb
import markdown
import pandas as pd
from tqdm.notebook import tqdm
from quest_scraper import MainQuestScraper, get_main_quest_list

In [2]:
(
    quest_names, quest_levels, quest_locations, quest_urls
) = get_main_quest_list(
    "https://witcher.fandom.com/wiki/The_Witcher_3_main_quests"
)

https://witcher.fandom.com


In [3]:
walkthroughs, journal_entries, objectives = [], [], []
for url in tqdm(quest_urls[:-1]):
    scraper = MainQuestScraper(quest_url=url)
    walkthroughs.append(scraper.scrape_walkthrough())
    journal_entries.append(scraper.scrape_journal_entry())
    objectives.append(scraper.scrape_objectives())

  0%|          | 0/60 [00:00<?, ?it/s]

In [4]:
data_frame = pd.DataFrame(
    data={
        "Quests": quest_names[:-1],
        "Quest_Levels": quest_levels[:-1],
        "Quest_Locations": quest_locations[:-1],
        "Quest_Page_Link": quest_urls[:-1],
        "Quest_Walkthrouughs": walkthroughs,
        "Journal_Entries": journal_entries,
        "Quest_Objectives": objectives
    }
)

In [5]:
data_frame.head()

,Quests,Quest_Levels,Quest_Locations,Quest_Page_Link,Quest_Walkthrouughs,Journal_Entries,Quest_Objectives
0,Kaer Morhen,0,Kaer Morhen,https://witcher.fandom.com/wiki/Kaer_Morhen_(q...,The game begins in the guest room of the witch...,Every story has to start somewhere. This one s...,* Use your Witcher Senses to find the key to t...
1,Lilac and Gooseberries,0,White Orchard,https://witcher.fandom.com/wiki/Lilac_and_Goos...,Upon waking from his dream of Yennefer and Cir...,Yennefer had contacted Geralt - finally. They ...,* Kill the ghouls.\n* Follow Vesemir.\n* Go to...
2,The Beast of White Orchard,3,White Orchard,https://witcher.fandom.com/wiki/The_Beast_of_W...,While the quest objectives indicate that you m...,The commander of the Nilfgaardian garrison in ...,* Ask the herbalist about buckthorn.\nFind som...
3,The Incident at White Orchard,2,White Orchard,https://witcher.fandom.com/wiki/The_Incident_a...,"After finishing The Beast of White Orchard, he...","Witchers should be neutral, avoid meddling in ...",* Kill the bandits.
4,Imperial Audience,2,Vizima,https://witcher.fandom.com/wiki/Imperial_Audience,This quest begins nice enough as you are bathe...,"Geralt had spent years looking for Yennefer, o...",* Get dressed and talk to the chamberlain.\n* ...


In [6]:
data_frame.to_csv("witcher-3-data/main_quests.csv", sep="\t")

In [7]:
with wandb.init(project="rpg-quest-generation", job_type="upload"):
    artifact = wandb.Artifact(name="witcher-3-quests", type="dataset")
    artifact.add_dir("witcher-3-data")
    wandb.log_artifact(artifact)

wandb: Currently logged in as: geekyrakshit. Use `wandb login --relogin` to force relogin


wandb: Adding directory to artifact (./witcher-3-data)... Done. 0.0s


In [8]:
data_frame.columns

Index(['Quests', 'Quest_Levels', 'Quest_Locations', 'Quest_Page_Link',
       'Quest_Walkthrouughs', 'Journal_Entries', 'Quest_Objectives'],
      dtype='object')

In [9]:
table_data = []
for idx in range(len(data_frame.index)):
    data = list(data_frame.iloc[idx])
    data[4] = wandb.Html(markdown.markdown(data[4]))
    data[5] = wandb.Html(markdown.markdown(data[5]))
    data[6] = wandb.Html(markdown.markdown(data[6]))
    table_data.append(data)
with wandb.init(project="rpg-quest-generation", job_type="upload"):
    wandb.log({"Main-Quests": wandb.Table(data=table_data, columns=list(data_frame.columns))})